In [1]:
import os
# os.chdir('../')
%pwd

'c:\\Users\\akish.pothuri\\python\\LangChain\\experiment'

In [1]:
import pyodbc
cnxn = pyodbc.connect(r'Driver=SQL Server;Server=GGKU4DELL1254;Database=projectcodefirst;Trusted_Connection=yes;')

In [2]:
cnxn.cursor()

In [3]:
cursor = cnxn.cursor()
cursor.execute("SELECT * FROM Employees")

In [4]:
row = cursor.fetchall()

In [5]:
row

[(1, 'Madhu', 1), (2, 'Anil', 2), (3, 'Amir', 3), (7, 'Akbar', 3)]

In [ ]:
# pip install google-generativeai
#pip install python-dotenv

In [7]:
import os
from langchain.llms import GooglePalm
from dotenv import load_dotenv 
load_dotenv()


True

In [10]:
palm_key = os.getenv('api_key')

In [11]:
llm = GooglePalm(google_api_key=palm_key, temperature=0.3)

In [12]:
code = llm("Code to linear regression in machine learning algorithms")

c:\Users\akish.pothuri\python\LangChain\SQl_Script_chatbot\chatbot\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [13]:
print(code)

```python
import numpy as np
import matplotlib.pyplot as plt

# Generate data
x = np.random.rand(100)
y = 2 * x + np.random.randn(100)

# Plot data
plt.scatter(x, y)
plt.show()

# Fit linear regression model
w, b = np.linalg.lstsq(x.reshape(-1, 1), y.reshape(-1, 1))[0]

# Plot fitted line
y_pred = w * x + b
plt.plot(x, y_pred)
plt.show()
```


In [14]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.sql_database import SQLDatabase
from langchain.prompts.chat import ChatPromptTemplate
from sqlalchemy import create_engine

In [15]:
# check the local ODBC driver and make sure it match with the traget database instance
import pyodbc
for driver in pyodbc.drivers():
    print(driver)

SQL Server
SQL Server Native Client 11.0
SQL Server Native Client RDA 11.0
ODBC Driver 17 for SQL Server


In [16]:
#windows authentication connection string.
db = SQLDatabase.from_uri(
    database_uri="mssql+pyodbc://GGKU4DELL1254/shopping?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes"
)

In [17]:
sql_toolkit=SQLDatabaseToolkit(db=db,llm=llm)
sql_toolkit.get_tools()

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000018817DBED00>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000018817DBED00>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000018817DBED00>),
 QuerySQLCheckerTool(description='Use this tool to 

In [18]:
print(db.get_table_names())

['Customer', 'OrderItem', 'Orders', 'Product', 'Supplier']


c:\Users\akish.pothuri\python\LangChain\SQl_Script_chatbot\chatbot\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `get_table_names` was deprecated in LangChain 0.0.1 and will be removed in 0.2.0. Use get_usable_table_name instead.
  warn_deprecated(


In [19]:
from langchain_experimental.sql import SQLDatabaseChain
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
qns1 = db_chain("what are the total number of customers")

c:\Users\akish.pothuri\python\LangChain\SQl_Script_chatbot\chatbot\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SQLDatabaseChain chain...
what are the total number of customers
SQLQuery:SELECT COUNT(*) FROM Customer
SQLResult: [(91,)]
Answer:91
> Finished chain.


In [20]:
from langchain_experimental.sql import SQLDatabaseChain
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
qns2 = db_chain("what are the names of the customers from germany courntry?")



> Entering new SQLDatabaseChain chain...
what are the names of the customers from germany courntry?
SQLQuery:SELECT FirstName, LastName FROM Customer WHERE Country = 'Germany'
SQLResult: [('Maria', 'Anders'), ('Hanna', 'Moos'), ('Sven', 'Ottlieb'), ('Peter', 'Franken'), ('Philip', 'Cramer'), ('Renate', 'Messner'), ('Alexander', 'Feuer'), ('Henriette', 'Pfalzheim'), ('Horst', 'Kloss'), ('Karin', 'Josephs'), ('Rita', 'Müller')]
Answer:Maria Anders, Hanna Moos, Sven Ottlieb, Peter Franken, Philip Cramer, Renate Messner, Alexander Feuer, Henriette Pfalzheim, Horst Kloss, Karin Josephs, Rita Müller
> Finished chain.


In [23]:
qns3 = db_chain("what is the name of Supplier providing more products?")




> Entering new SQLDatabaseChain chain...
what is the name of Supplier providing more products?
SQLQuery:SELECT CompanyName FROM Supplier s JOIN Product p ON s.Id = p.SupplierId GROUP BY CompanyName ORDER BY COUNT(*) DESC
SQLResult: [('Pavlova, Ltd.',), ('Plutzer Lebensmittelgroßmärkte AG',), ('Specialty Biscuits, Ltd.',), ('New Orleans Cajun Delights',), ('Norske Meierier',), ('Formaggi Fortini s.r.l.',), ("Mayumi's",), ('Bigfoot Breweries',), ('Exotic Liquids',), ("G'day, Mate",), ("Grandma Kelly's Homestead",), ('Heli Süßwaren GmbH & Co. KG',), ('Karkki Oy',), ('Leka Trading',), ('Svensk Sjöföda AB',), ('Tokyo Traders',), ('Zaanse Snoepfabriek',), ('PB Knäckebröd AB',), ('Aux joyeux ecclésiastiques',), ('Lyngbysild',), ('Ma Maison',), ("Forêts d'érables",), ("Cooperativa de Quesos 'Las Cabras'",), ('New England Seafood Cannery',), ('Gai pâturage',), ('Pasta Buttini s.r.l.',), ('Nord-Ost-Fisch Handelsgesellschaft mbH',), ('Escargots Nouveaux',), ('Refrescos Americanas LTDA',)]
Answe

In [24]:
qns3['result']

'Pavlova, Ltd.'

In [26]:
qns4 = db_chain("what is the name of customer ordered maximum quantity according to the data in the database?")



> Entering new SQLDatabaseChain chain...
what is the name of customer ordered maximum quantity according to the data in the database?
SQLQuery:SELECT TOP 1 c.FirstName + ' ' + c.LastName AS CustomerName 
FROM Customer AS c 
INNER JOIN Orders AS o ON c.Id = o.CustomerId 
INNER JOIN OrderItem AS i ON o.Id = i.OrderId 
GROUP BY c.Id, c.FirstName, c.LastName 
ORDER BY SUM(i.Quantity) DESC
SQLResult: [('Jose Pavarotti',)]
Answer:Jose Pavarotti
> Finished chain.


In [39]:
qns5 = db_chain("Provide me names of customers according to their respective countries")
# print(qns5)



> Entering new SQLDatabaseChain chain...
Provide me names of customers according to their respective countries
SQLQuery:SELECT c.FirstName, c.LastName, c.Country FROM Customer c
SQLResult: [('Maria', 'Anders', 'Germany'), ('Ana', 'Trujillo', 'Mexico'), ('Antonio', 'Moreno', 'Mexico'), ('Thomas', 'Hardy', 'UK'), ('Christina', 'Berglund', 'Sweden'), ('Hanna', 'Moos', 'Germany'), ('Frédérique', 'Citeaux', 'France'), ('Martín', 'Sommer', 'Spain'), ('Laurence', 'Lebihan', 'France'), ('Elizabeth', 'Lincoln', 'Canada'), ('Victoria', 'Ashworth', 'UK'), ('Patricio', 'Simpson', 'Argentina'), ('Francisco', 'Chang', 'Mexico'), ('Yang', 'Wang', 'Switzerland'), ('Pedro', 'Afonso', 'Brazil'), ('Elizabeth', 'Brown', 'UK'), ('Sven', 'Ottlieb', 'Germany'), ('Janine', 'Labrune', 'France'), ('Ann', 'Devon', 'UK'), ('Roland', 'Mendel', 'Austria'), ('Aria', 'Cruz', 'Brazil'), ('Diego', 'Roel', 'Spain'), ('Martine', 'Rancé', 'France'), ('Maria', 'Larsson', 'Sweden'), ('Peter', 'Franken', 'Germany'), ('Cari

In [41]:
print(qns5['result'])

Maria Anders (Germany), Ana Trujillo (Mexico), Antonio Moreno (Mexico), Thomas Hardy (UK), Christina Berglund (Sweden), Hanna Moos (Germany), Frédérique Citeaux (France), Martín Sommer (Spain), Laurence Lebihan (France), Elizabeth Lincoln (Canada), Victoria Ashworth (UK), Patricio Simpson (Argentina), Francisco Chang (Mexico), Yang Wang (Switzerland), Pedro Afonso (Brazil), Elizabeth Brown (UK), Sven Ottlieb (Germany), Janine Labrune (France), Ann Devon (UK), Roland Mendel (Austria), Aria Cruz (Brazil), Diego Roel (Spain), Martine Rancé (France), Maria Larsson (Sweden), Peter Franken (Germany), Carine Schmitt (France), Paolo Accorti (Italy), Lino Rodriguez (Portugal), Eduardo Saavedra (Spain), José Pedro Freyre (Spain), André Fonseca (Brazil), Howard Snyder (USA), Manuel Pereira (Venezuela), Mario Pontes (Brazil), Carlos Hernández (Venezuela), Yoshi Latimer (USA), Patricia McKenna (Ireland), Helen Bennett (UK), Philip Cramer (Germany), Daniel Tonini (France), Annette Roulet (France), Y

In [42]:
qns6 = db_chain.run("SELECT Country, FirstName + ' ' + LastName FROM Customer GROUP BY Country, FirstName, LastName;")



> Entering new SQLDatabaseChain chain...
SELECT Country, FirstName + ' ' + LastName FROM Customer GROUP BY Country, FirstName, LastName;
SQLQuery:SELECT Country, FirstName + ' ' + LastName FROM Customer GROUP BY Country, FirstName, LastName
SQLResult: [('Argentina', 'Patricio Simpson'), ('Argentina', 'Sergio Gutiérrez'), ('Argentina', 'Yvonne Moncada'), ('Austria', 'Georg Pipps'), ('Austria', 'Roland Mendel'), ('Belgium', 'Catherine Dewey'), ('Belgium', 'Pascale Cartrain'), ('Brazil', 'Anabela Domingues'), ('Brazil', 'André Fonseca'), ('Brazil', 'Aria Cruz'), ('Brazil', 'Bernardo Batista'), ('Brazil', 'Janete Limeira'), ('Brazil', 'Lúcia Carvalho'), ('Brazil', 'Mario Pontes'), ('Brazil', 'Paula Parente'), ('Brazil', 'Pedro Afonso'), ('Canada', 'Elizabeth Lincoln'), ('Canada', 'Jean Fresnière'), ('Canada', 'Yoshi Tannamuri'), ('Denmark', 'Jytte Petersen'), ('Denmark', 'Palle Ibsen'), ('Finland', 'Matti Karttunen'), ('Finland', 'Pirkko Koskitalo'), ('France', 'Annette Roulet'), ('Franc

In [29]:
qns7 = db_chain("Give me the date on we got maximum amount?")



> Entering new SQLDatabaseChain chain...
Give me the date on we got maximum amount?
SQLQuery:

Retrying langchain_community.llms.google_palm.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


SELECT OrderDate FROM Orders ORDER BY TotalAmount DESC OFFSET 0 ROWS FETCH NEXT 1 ROWS ONLY
SQLResult: [(datetime.datetime(2014, 2, 2, 0, 0),)]
Answer:2014-02-02
> Finished chain.


In [30]:
qns8 = db_chain("Give me the date on we got minimum amount?")



> Entering new SQLDatabaseChain chain...
Give me the date on we got minimum amount?
SQLQuery:

Retrying langchain_community.llms.google_palm.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


SELECT MIN(OrderDate) FROM Orders
SQLResult: [(datetime.datetime(2012, 7, 4, 0, 0),)]
Answer:

Retrying langchain_community.llms.google_palm.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.
Retrying langchain_community.llms.google_palm.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


2012-07-04
> Finished chain.


## Few Shot

In [ ]:
few_shots = [
    {
        'Question' : ""
    }
]